In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import glob

In [2]:
base_path  = Path('./EPA_GHGI_2025/2025_Main_Report_Tables') #draft 2025 report (1990-2023)
# dataset = 'EPA_GHGI_DataExplorer/Data_Explorer_output.csv' #dataexplorer only goes to the 2024 report 1990-2022

In [3]:
sectors_final = ['Energy', 'Manufacturing', 'Land Management', 'Waste Management']

In [4]:
sectors_EPA = ['Energy', 'Industrial Processes and Product Use', 'Agriculture', 'Waste', 'Total Gross Emissionsd  (Sources)', 
'LULUCF Sector Net Totae', 'Net Emission (Sources and Sinks)f']
## [SIC] TYPOS IN THE DRAFT REPORT TABLES

## Functions

In [5]:
def parse_report_tables(glob_path, notes_start_row):
    path = list(base_path.glob(glob_path))[0]

    # Notes and title
    meta = pd.read_csv(path)
    title = meta.columns[0]
    notes = meta.iloc[notes_start_row:,0].tolist()

    # Data
    df_clean = pd.read_csv(path, header=1)
    df_clean.set_index(df_clean.columns[0],inplace=True)
    df_clean = df_clean.iloc[:notes_start_row-1,:]
    return title, notes, df_clean

## Emission Types for Each Sector

### Import data and parse

In [6]:
emit_glob = "Chapter 2*/Table 2-3.csv"
title_emit, notes_emit, df_emit = parse_report_tables(emit_glob, 64)
print(title_emit,*notes_emit, sep='\n')

Table 2-3:  Recent Trends in U.S. Greenhouse Gas Emissions and Sinks by Inventory Sector/Category (MMT CO2 Eq.)
+ Does not exceed 0.05 MMT CO2 Eq.
a Includes CH4 and N2O emissions from fuel combustion.
b Emissions from biomass and biofuel consumption are not included specifically in summing Energy sector totals. Net carbon fluxes from changes in biogenic carbon reservoirs are accounted for in the estimates for LULUCF.
c Emissions from international bunker fuels are not included in totals.
d SF6 and PFCs from other product use category includes the use of HFCs and NF3.
e Total emissions without LULUCF.
f LULUCF emissions of CH4 and N2O are reported separately from gross emissions totals. LULUCF emissions include the CH4 and N2O emissions reported for peatlands remaining peatlands, forest fires, drained organic soils, grassland fires, and coastal wetlands remaining coastal wetlands; CH4 emissions from land converted to coastal wetlands, flooded land remaining flooded land, and land conve

In [7]:
# df_emit

In [8]:
# WHATEVER VERSION OF THESE TABLES ARE THE DRAFTS NOT THE FINAL REPORT TABLES! TYPOS!!!!!!
# df_clean.index.tolist() 

In [9]:
# Need to split by subheaders:
sector_col = []
current_sector = None

for idx, row in df_emit.iterrows():
    if idx in sectors_EPA:
        current_sector = idx
        sector_col.append(current_sector)
    else:
        sector_col.append(current_sector)

df_emit['Sector'] = sector_col
new_columns = pd.MultiIndex.from_arrays([df_emit['Sector'], df_emit.index.get_level_values(0)])
# new_columns[0]

df_emit.set_index(new_columns, inplace=True)

# Select 2021-2023
df_emit_sub = df_emit.iloc[:, -4:-1]
df_emit_sub

2021  \
Sector                            Inventory Sector/Category                        
Energy                            Energy                             5170.069933   
                                  Fossil Fuel Combustion             4654.628938   
                                  Natural Gas Systems                 210.375072   
                                  Non-Energy Use of Fuels             111.717610   
                                  Petroleum Systems                    69.212623   
...                                                                          ...   
LULUCF Sector Net Totae           Cropland                              3.001371   
                                  Grassland                            31.069619   
                                  Wetlands                             40.327585   
                                  Settlements                         -52.356995   
Net Emission (Sources and Sinks)f Net Emission (Sources and Sinks)f  5370.960988   

                                                                            2022  \
Sector                            Inventory Sector/Category                        
Energy                            Energy                             5196.182383   
                                  Fossil Fuel Combustion             4702.769192   
                                  Natural Gas Systems                 209.276434   
                                  Non-Energy Use of Fuels             101.697181   
                                  Petroleum Systems                    58.392610   
...                                                                          ...   
LULUCF Sector Net Totae           Cropland                              3.459028   
                                  Grassland                            34.642917   
                                  Wetlands                             40.370137   
                                  Settlements                         -52.343588   
Net Emission (Sources and Sinks)f Net Emission (Sources and Sinks)f  5438.749818   

                                                                            2023  
Sector                            Inventory Sector/Category                       
Energy                            Energy                             5050.356406  
                                  Fossil Fuel Combustion             4559.379170  
                                  Natural Gas Systems                 200.135715  
                                  Non-Energy Use of Fuels             107.069482  
                                  Petroleum Systems                    61.321741  
...                                                                          ...  
LULUCF Sector Net Totae           Cropland                              5.042523  
                                  Grassland                            43.605806  
                                  Wetlands                             40.384680  
                                  Settlements                         -51.877617  
Net Emission (Sources and Sinks)f Net Emission (Sources and Sinks)f  5257.374693  

[63 rows x 3 columns]

In [10]:
#Check the typos were all resolved
# df_emit_sub.index.values

### Reorganize into sectors_final

In [11]:
sectors_final

['Energy', 'Manufacturing', 'Land Management', 'Waste Management']

In [12]:
# Energy
df_E = df_emit_sub.loc[sectors_EPA[0]].drop([sectors_EPA[0]])
# df_E["2021-2023 Average"] = df_E.mean(axis=1)

# Manufacturing
df_M = df_emit_sub.loc[sectors_EPA[1]].drop([sectors_EPA[1]])
# df_M["2021-2023 Average"] = df_M.mean(axis=1)

# Land mgmt
df_L1 = df_emit_sub.loc[sectors_EPA[2]].drop([sectors_EPA[2]])
df_L2 = df_emit_sub.loc[sectors_EPA[5]].drop([sectors_EPA[5]])
df_L = pd.concat([df_L1, df_L2])
# df_L["2021-2023 Average"] = df_L.mean(axis=1)


# Waste mgmt
df_W = df_emit_sub.loc[sectors_EPA[3]].drop([sectors_EPA[3]])
# df_W["2021-2023 Average"] = df_W.mean(axis=1)


# Store dataframes in a dictionary
dataframes_emittype = {
    sectors_final[0]: df_E,
    sectors_final[1]: df_M,
    sectors_final[2]: df_L,
    sectors_final[3]: df_W,
    
}

### Calculate average and percents and write out

In [16]:
# Write to csv:
for key, df in dataframes_emittype.items():
    df["2021-2023 Average"] = df.mean(axis=1)
    if key != sectors_final[2]:
        df["Percent"] = (df.iloc[:,-1] / df.iloc[:,-1].sum())*100 # Calculate percent for all but land management
    output=f'output_tables/{key}_Sectors.csv'
    df.to_csv(output)

## Emission Gas Type for Each Sector

### Energy

In [17]:
energy_gas_glob = "Chapter 2*/Table 2-4.csv"
title_E_gas, notes_E_gas, df_E_gas = parse_report_tables(energy_gas_glob, 39)
print(title_E_gas,*notes_E_gas, sep='\n')

Table 2-4:  Emissions from Energy by Gas (MMT CO2 Eq.) [2]
+ Does not exceed 0.05 MMT CO2 Eq.
a Emissions from biomass and biofuel consumption are not included specifically in Energy sector totals. Net carbon fluxes from changes in biogenic carbon reservoirs are accounted for in the estimates for LULUCF.
b Emissions from international bunker fuels are not included in totals. These values are presented for informational purposes only, in line with the 2006 IPCC Guidelines, the national inventory reporting guidelines.
Note: Totals may not sum due to independent rounding. 


In [18]:
gas_energy = ['CO2','CH4', 'N2O']
df_E_gas_sub = df_E_gas.loc[gas_energy].iloc[:, -4:-1]
df_E_gas_sub

,2021,2022,2023
Gas/Source,,,
CO2,4841.185868,4877.978213,4742.335937
CH4,289.633031,278.650522,271.853658
N2O,39.251035,39.553648,36.166811


### Manufacturing

In [19]:
man_gas_glob = "Chapter 2*/Table 2-6.csv"
title_M_gas, notes_M_gas, df_M_gas = parse_report_tables(man_gas_glob, 58)
print(title_M_gas,*notes_M_gas, sep='\n')

Table 2-6:  Emissions from Industrial Processes and Product Use (MMT CO2 Eq.)
+ Does not exceed 0.05 MMT CO2 Eq.
a Small amounts of PFC emissions from this source are included under HFCs due to confidential business information.
Note: Totals may not sum due to independent rounding.


In [20]:
gas_man = ['CO2','CH4', 'N2O', 'HFCs', 'PFCs', 'SF6', 'NF3']
df_M_gas_sub = df_M_gas.loc[gas_man].iloc[:, -4:-1]
df_M_gas_sub

,2021,2022,2023
,,,
CO2,171.433258,169.037333,165.532915
CH4,0.035034,0.035898,0.036106
N2O,19.718749,16.078250,14.859472
HFCs,184.268791,189.518123,190.968214
PFCs,6.313880,6.524678,5.771697
SF6,7.996214,7.233500,7.724037
NF3,1.127476,1.123111,0.815250


### Land Management (this needs work for the land use/change table - how do I extract gases)

### Agriculture

In [21]:
land1_gas_glob = "Chapter 2*/Table 2-7.csv"
title_L1_gas, notes_L1_gas, df_L1_gas = parse_report_tables(land1_gas_glob, 14)
print(title_L1_gas,*notes_L1_gas, sep='\n')

Table 2-7:  Emissions from Agriculture (MMT CO2 Eq.)
Note: Totals may not sum due to independent rounding.


In [22]:
gas_land1 = ['CO2','CH4', 'N2O']
df_L1_gas_sub = df_L1_gas.loc[gas_land1].iloc[:, -4:-1]
df_L1_gas_sub

,2021,2022,2023
Gas/Source,,,
CO2,7.491839,8.387506,10.537832
CH4,282.037265,275.897637,271.568195
N2O,316.298203,308.971066,313.291939


### Land Use, Land-Use Change, Forestry (LULCF)

In [23]:
land2_gas_glob = "Chapter 6*/Table 6-2.csv"
title_L2_gas, _, df_L2_gas = parse_report_tables(land2_gas_glob, 32) # The notes are in another column

In [24]:
metaL2 = pd.read_csv(list(base_path.glob(land2_gas_glob))[0])
notes_L2_gas = metaL2.iloc[32:,:2].values.tolist()
print(title_L2_gas,*notes_L2_gas, sep='\n')

Table 6-2:  Emissions and Removals from Land Use, Land-Use Change, and Forestry by Gas (MMT CO2 Eq.)
['+ ', 'Absolute value does not exceed 0.05 MMT CO2 Eq.']
['a', 'LULUCF carbon stock change is the net carbon stock change from the following categories: forest land remaining forest land, land converted to forest land, cropland remaining cropland, land converted to cropland, grassland remaining grassland, land converted to grassland, wetlands remaining wetlands, land converted to wetlands, settlements remaining settlements, and land converted to settlements.']
['b', 'Estimates include CH4 and N2O emissions from fires on both forest land remaining forest land and land converted to forest land.']
['c', 'Estimates include CH4 and N2O emissions from drained organic soils on both forest land remaining forest land and land converted to forest land.']
['d', 'Estimates include CH4 and N2O emissions from fires on both grassland remaining grassland and land converted to grassland.']
['e', 'Estim

In [25]:
gas_land2 = ['Carbon Stock Change (CO2)a','CH4', 'N2O']
df_L2_gas_sub = df_L2_gas.loc[gas_land2].iloc[:, -3:]
df_L2_gas_sub = df_L2_gas_sub.rename(index={'Carbon Stock Change (CO2)a':'CO2'})

### Combine L1 and L2

In [26]:
df_L_gas_sub = df_L1_gas_sub + df_L2_gas_sub
df_L_gas_sub

,2021,2022,2023
Gas/Source,,,
CO2,-1036.308161,-965.512494,-989.962168
CH4,349.837265,335.497637,326.268195
N2O,329.398203,317.971066,319.191939


### Waste Management

In [27]:
man_was_glob = "Chapter 2*/Table 2-9.csv"
title_W_gas, notes_W_gas, df_W_gas = parse_report_tables(man_was_glob, 10)
print(title_W_gas,*notes_W_gas, sep='\n')

Table 2-9:  Emissions from Waste (MMT CO2 Eq.)
+ Does not exceed 0.05 MMT CO2 Eq.
Note: Totals may not sum due to independent rounding.


In [28]:
gas_was = ['CH4', 'N2O']
df_W_gas_sub = df_W_gas.loc[gas_was].iloc[:, -4:-1]
df_W_gas_sub

,2021,2022,2023
Gas/Source,,,
CH4,143.926410,142.180520,143.202959
N2O,23.094419,22.920659,22.646561


### Make dictionary of dataframes by emittype

In [29]:
dataframes_gastype = {
    sectors_final[0]: df_E_gas_sub,
    sectors_final[1]: df_M_gas_sub,
    sectors_final[2]: df_L_gas_sub,
    sectors_final[3]: df_W_gas_sub,
    
}

In [ ]:
# for df in dataframes_gastype.values():
#     # print(df)
#     df["2021-2023 Average"] = df.mean(axis=1)
#     df["Percent"] = (df.iloc[:,-1] / df.iloc[:,-1].sum())*100

In [30]:
# Write to csv:
for key, df in dataframes_gastype.items():
    df["2021-2023 Average"] = df.mean(axis=1)
    if key != sectors_final[2]:
        df["Percent"] = (df.iloc[:,-1] / df.iloc[:,-1].sum())*100 # Calculate percent for all but land management
    output=f'output_tables/{key}_Gases.csv'
    df.to_csv(output)

In [ ]:
# dataframes_emittype
# dataframes_gastype

## Plot

In [ ]:
# fig.suptitle('2021-2023 Average Emissions',y=0.95)
for (idx, (sector, df1)), (idx, (sector, df2)) in zip(enumerate(dataframes_emittype.items()), enumerate(dataframes_gastype.items())):
    # Skip land use
    if idx==2:
        continue
    else:
        fig, ax = plt.subplots(1, 2, figsize=(12,10), 
                       constrained_layout=True
                      )
        fig.suptitle(sector, y=0.9)
        
        df1_avg = df1.mean(axis=1).to_frame('2021-2023 Average')
        df1_avg["Percent"] = (df1_avg.iloc[:, 0] / df1_avg.iloc[:, 0].sum()).apply(lambda x: f"{x:.3%}")
        # print(df1_avg)
        ax[0].pie(df1_avg.iloc[:,0], 
                           startangle=90,)
        ax[0].legend(df1_avg.index,
                    # loc="best",
                    )
        ax[0].set_title('Emission Type')

        df2_avg = df2.mean(axis=1).to_frame('2021-2023 Average')
        df2_avg["Percent"] = (df2_avg.iloc[:, 0] / df2_avg.iloc[:, 0].sum()).apply(lambda x: f"{x:.3%}")
        ax[1].pie(df2_avg.iloc[:,0], 
                           startangle=90,)
        ax[1].legend(df2_avg.index)
        
        ax[1].set_title('Gas Type')

                # # axis.legend(labels, 
        # #             fontsize=9,
        # #             bbox_to_anchor=(2, 0.5)
        # #            )
    
        # # labels = [f"{label}: {percent}" for label, percent in zip(df_avg.index, df_avg['Percent'])]
        # # axis = ax.flat[idx]
        # # axis.pie(df_avg.iloc[:,0], 
        # #                    startangle=90,
        # #                   )
        # # axis.legend(labels, 
        # #             loc="center right", 
        # #             fontsize=9,
        # #             bbox_to_anchor=(2, 0.5)$ 
        # #            )
